# **Lorem Ipsum Dolor Sit Amet**

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec vestibulum tortor tortor, sed lobortis velit pharetra et. Mauris at lectus quis lorem viverra cursus. Praesent a efficitur lectus, vitae aliquet urna. Phasellus aliquam felis non consequat eleifend. Nulla ut feugiat ex. Suspendisse non justo vel neque dignissim vestibulum. Praesent feugiat vel nisi eu ullamcorper. Sed tincidunt mollis ex. 

**Authors:**
- [Adrian Flores](https://github.com/adrianRFlores)
- [Andrea Ramirez](https://github.com/Andrea-gt)

***

## **(1) Import Libraries** ⬇️

In [52]:
#!pip install unidecode

In [53]:
# Data manipulation and visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from unidecode import unidecode
import json
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

# Standard libraries
import warnings
warnings.filterwarnings('ignore')

# ===== ===== Reproducibility Seed ===== =====
# Set a fixed seed for the random number generator for reproducibility
random_state = 42

# Set matplotlib inline
%matplotlib inline

# Set default figure size
plt.rcParams['figure.figsize'] = (6, 4)

# Define custom color palette
palette = sns.color_palette("viridis", 12)

# Set the style of seaborn
sns.set(style="whitegrid")

## **(2) Data Upload** 📄

In [54]:
# Read the .txt file 
with open('data/tweets.txt', 'r', encoding='utf-16') as file:
    lines = file.readlines()

In [55]:
# List to hold extracted tweet data
tweet_data = []

# Loop through each line, assuming each line is a separate JSON object
for line in lines:
    # Parse the line as JSON
    data = json.loads(line.strip())
    
    # Extract the relevant information
    tweet_entry = {
        "tweet_date": pd.to_datetime(data["date"]).date(),
        "user_username": data["user"]["username"],
        "mentioned_users": ", ".join([user["username"] for user in data["mentionedUsers"]]),
        "reply_count": data["replyCount"],
        "retweet_count": data["retweetCount"],
        "like_count": data["likeCount"],
        "quote_count": data["quoteCount"],
        'tweet': data['rawContent'],
        "hashtags": ", ".join(data["hashtags"])
    }
    
    # Append the tweet data to the list
    tweet_data.append(tweet_entry)

In [56]:
# Create the DataFrame
df = pd.DataFrame(tweet_data)

# Display the DataFrame
df.head(3)

,tweet_date,user_username,mentioned_users,reply_count,retweet_count,like_count,quote_count,tweet,hashtags
0,2024-09-12,La_ReVoluZzion,"usembassyguate, 48CantonesToto, USAIDGuate, UE...",0,0,0,0,"_\nConfirmado Compañeres,\n\nEl impuesto por l...",
1,2024-09-12,XelaNewsGt,,12,80,142,15,#URGENTE Lo que los medios #faferos no informa...,"URGENTE, faferos, BernardoArévalo, NebajQuiché..."
2,2024-09-12,M24095273,"IvanDuque, BArevalodeLeon",0,0,0,0,@IvanDuque @BArevalodeLeon Con que usaste PEGA...,


**Observaciones 💡 -->**

Notar que ya se han llevado a cabo algunas acciones importantes de pre-procesamiento con el objetivo de optimizar el conjunto de datos, en el contexto del análisis de redes sociales (esto puede variar según el propósito del dataset). A continuación, se detallan los pasos realizados:

> - **Reducción de columnas:** Se han eliminado aquellas columnas que no aportan valor relevante para el análisis de redes sociales, dejando únicamente las siguientes variables: tweet_date, user_username, mentioned_users, reply_count, retweet_count, like_count, quote_count, tweet y hashtags. La finalidad es reducir la dimensionalidad del conjunto de datos, eliminando características que solo añaden ruido innecesario.

> - **Formateo de fecha:** La columna tweet_date, que originalmente incluía tanto la fecha como la hora, ha sido simplificada para contener únicamente el día, mes y año. Este ajuste es especialmente útil dado que la precisión horaria no es relevante para el análisis, contribuyendo así a la homogeneización del formato de los datos.

## **(3) Exploratory Analysis** 🔎

### **(1) Descripción General de los Datos**

In [57]:
# Print the number of records in the DataFrame
print("The given dataset has", df.shape[0], "registers and", df.shape[1], "columns.")

The given dataset has 5019 registers and 9 columns.


**Observaciones 💡 -->**

> - El conjunto de datos actual cuenta con 5019 registros y las 9 columnas previamente mencionadas, indicando una dimensión relativamente pequeña. Este conjunto de datos consta de tweets de la plataforma `X`, que de alguna u otra forma están ligados a el usuario `@BArevalodeLeon`. Cada uno de los 7613 registros representa un tweet único, mientras que las 9 columnas corresponden a diferentes características o variables medidas para cada observación, incluyendo el texto y fecha de un tweet, así como las estadísticas de este.

In [58]:
# Basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5019 entries, 0 to 5018
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tweet_date       5019 non-null   object
 1   user_username    5019 non-null   object
 2   mentioned_users  5019 non-null   object
 3   reply_count      5019 non-null   int64 
 4   retweet_count    5019 non-null   int64 
 5   like_count       5019 non-null   int64 
 6   quote_count      5019 non-null   int64 
 7   tweet            5019 non-null   object
 8   hashtags         5019 non-null   object
dtypes: int64(4), object(5)
memory usage: 353.0+ KB


**Observaciones 💡 -->**

> - Por el momento, no se ha detectado la presencia de valores faltantes  en ninguna de las columnas. Sin embargo, es evidente que sí existe información ausente en algunos registros, lo que sugiere que será necesario realizar un preprocesamiento adicional para identificar adecuadamente estos datos faltantes. Una vez detectados, será importante aplicar técnicas de imputación que permitan completar o manejar estos valores de manera adecuada.

El conjunto de datos contiene 9 columnas o características, las cuales se describen a continuación:

- **tweet_date**: La fecha en la que se publicó el tweet.
- **user_username**: El nombre de usuario de la cuenta que publicó el tweet. 
- **mentioned_users**: Los usuarios mencionados en el tweet. 
- **reply_count**: El número de respuestas que recibió el tweet. 
- **retweet_count**: La cantidad de veces que el tweet fue retuiteado. 
- **like_count**: El número de "me gusta" que recibió el tweet. 
- **quote_count**: La cantidad de veces que el tweet fue citado. 
- **tweet**: El contenido textual del tweet.
- **hashtags**: Los hashtags incluidos en el tweet.

Las características incluyen tanto datos de tipo numérico (`int64`) como de tipo texto (`object`).

### **(2) Clasificación de las Variables**

| **Nombre de la columna** | **Descripción**                                                  | **Clasificación**                   |
|--------------------------|------------------------------------------------------------------|-------------------------------------|
| `tweet_date`             | Fecha en la que se publicó el tweet                              | Cualitativa (descriptiva)           |
| `user_username`          | Nombre de usuario de la cuenta que publicó el tweet              | Cualitativa (descriptiva)           |
| `mentioned_users`        | Usuarios mencionados en el tweet                                 | Cualitativa (descriptiva)           |
| `reply_count`            | Número de respuestas que recibió el tweet                        | Cuantitativa (discreta)             |
| `retweet_count`          | Número de veces que fue retuiteado el tweet                      | Cuantitativa (discreta)             |
| `like_count`             | Número de "me gusta" que recibió el tweet                        | Cuantitativa (discreta)             |
| `quote_count`            | Número de veces que fue citado el tweet                          | Cuantitativa (discreta)             |
| `tweet`                  | Contenido textual del tweet                                      | Cualitativa (descriptiva)           |
| `hashtags`               | Hashtags incluidos en el tweet                                   | Cualitativa (descriptiva)           |


**Observaciones 💡 -->**

> - El conjunto de datos posee 5 variables cualitativas descriptivas.
> - Las últimas 4 variables del conjunto de datos son de tipo cuantitativo.

### **(3) Exploración y Limpieza Inicial de los Datos**

#### **(1) Preprocesamiento de los Datos**

In [59]:
# Download the NLTK stopwords if not already available
nltk.download('stopwords')
# Initialize the PorterStemmer
stemmer = SnowballStemmer('spanish')

[nltk_data] Downloading package stopwords to /home/andrea/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
# Get the list of stopwords from NLTK
stop_words = set(stopwords.words('spanish'))

In [61]:
# Function to remove stopwords and apply stemming
def preprocess_text(text):
    # Tokenize the text
    words = text.split()
    # Remove stopwords and apply stemming
    processed_words = [word for word in words if word not in stop_words]
    # Reassemble the text
    return ' '.join(processed_words)

**Columna `tweet` -->** 

In [62]:
# Convert all entries to strings
df['tweet'] = df['tweet'].astype(str)
# Remove leading/trailing whitespaces
df['tweet'] = df['tweet'].str.strip()
# Remove accents and/or special characters
df['tweet'] = df['tweet'].apply(unidecode)
# Convert to lowercase
df['tweet'] = df['tweet'].str.lower()
# Remove usernames from the tweet text
df['tweet'] = df['tweet'].str.replace(r'@\w+', '', regex=True).str.strip()
# Remove URLs
df['tweet'] = df['tweet'].str.replace(r'http\S+|www\S+|https\S+', '', case=False, regex=True)
# Keeping letters, numbers, and spaces
df['tweet'] = df['tweet'].str.replace(r'[^\w\s]', '', regex=True)
# Remove extra spaces
df['tweet'] = df['tweet'].str.replace(r'\s+', ' ', regex=True)
# Apply preprocessing (stopwords removal and stemming)
df['tweet'] = df['tweet'].apply(preprocess_text)

**Columna `hashtags` -->** 